In [1]:
import numpy as np
import pandas as pd

from scipy.spatial.distance import cdist, euclidean
from scipy.optimize import linear_sum_assignment

In [19]:
pred_df = pd.read_csv("files/zeroes.csv", index_col=0)
gt_df = pd.read_csv("files/threeHB.csv", index_col=0)
gt_df.head()

,t,hexbug,x,y
0,0,0,562.774938,700.299740
1,1,0,579.863030,601.057360
2,2,0,572.633453,496.557106
3,3,0,530.570457,402.572600
4,4,0,487.850227,342.764278


In [3]:
PENALTY_MORE_THAN_FIVE_HEXBUGS = 5_000
PENALTY_ZERO_HEXBUGS = 5_000
PENALTY_N_WRONG_HEXBUGS = 1_000 # only applies if the amount of predicted hexbugs is between 1 and 5

In [18]:
for idx, elem in enumerate(pred_df.keys()):
    print(type(pred_df[elem]))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [16]:
if 'hexbug' not in pred_df.keys():
    print("No hexbug column in prediction file")

No hexbug column in prediction file


In [10]:
def get_score(path_to_upload: str, path_to_gt:str):
    # Check ths submitted json data

    #check_guidelines(pred_json)
    
    final_score = 0

    # Load the data
    pred_df = pd.read_csv(path_to_upload, index_col=0)
    gt_df = pd.read_csv(path_to_gt, index_col=0)

    # Determine how many frames we have and how many hexbugs are used in the video of the ground truth
    n_frames = gt_df['t'].max()
    n_hexbugs_gt = gt_df['hexbug'].max()    
    n_hexbugs_pred = pred_df['hexbug'].max()
            
    # Apply penalties if the amount of predicted hexbugs is wrong
    if n_hexbugs_pred < 1:
        final_score += PENALTY_ZERO_HEXBUGS
    elif n_hexbugs_pred > 5:
        final_score += PENALTY_MORE_THAN_FIVE_HEXBUGS
    else:
        final_score += (np.abs(n_hexbugs_pred - n_hexbugs_gt) * PENALTY_N_WRONG_HEXBUGS)  
    
    # Match the predicted hexbugs in the first frame with the ground truth 
    # --> the matching IDs are used in all the following frames
    distance_matrix = cdist(list(gt_df[gt_df['t'] == 0][['x', 'y']].values), list(pred_df[pred_df['t'] == 0][['x', 'y']].values))
    row_ind, col_ind = linear_sum_assignment(distance_matrix)  # Hungarian algorithm

    matched_ids = {}
    for i, j in zip(row_ind, col_ind):
        matched_ids[f"{i}"] = j
    
    # Iterate over each frame and calculate distances
    for i in range(n_frames):
        frame_gt_df = gt_df[gt_df['t'] == i]
        frame_pred_df = pred_df[pred_df['t'] == i]
        for j in range(n_hexbugs_gt):
            row_gt = frame_gt_df[frame_gt_df['hexbug'] == j]
            row_pred = frame_pred_df[frame_pred_df['hexbug'] == matched_ids[str(j)]]
            score = euclidean(list(row_gt[['x', 'y']].values[0]), list(row_pred[['x', 'y']].values[0]))
            final_score += score

    return final_score

In [21]:
print("Score: ", get_score("files/training053.csv", "files/training056.csv"))

Score:  5000


In [22]:
print("Score: ", get_score("files/zeroes.csv", "files/threeHB.csv"))

Score:  191696.34032764478
